In [1]:
import librosa
import pydub

Librosa basic example (from documentation, my comments)

In [10]:
#OPENS EXAMPLE
filename = librosa.example('nutcracker')

#sampling_rate=number of samples per second of audi
#waveform saved as one-dimensional NumPy floating point array
waveform, sampling_rate = librosa.load(filename)

#tempo in BPM
# beat_frames - frames with detected beats, it's going to be converted. Some strange things happening there - worth double checking documentation
tempo, beat_frames = librosa.beat.beat_track(y=waveform, sr=sampling_rate)             
print(f'Estimated tempo: {tempo} beats per minute')

# beat_times = an array of timestamps (in seconds) corresponding to detected beats.
beat_times = librosa.frames_to_time(beat_frames, sr=sampling_rate)

Estimated tempo: [107.66601562] beats per minute


Original advance example and comments (I haven't went through it yet) from librosa documentation            

In [11]:
# Feature extraction example
import numpy as np
import librosa

# Load the example clip
y, sr = librosa.load(librosa.ex('nutcracker'))

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 512

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr)

# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])